# Topic Modelling and Sentiment Analysis of Jira Support Tickets

&copy; Anika Nissen

Inspired by the following tutorial: https://www.kaggle.com/code/dabrom55/abstracts-clustering-lda-nmf-scibert-embeddings/notebook


In [1]:
#TODO: Check gensimNMF Model and the coherence score.... seems to be easier 
# https://notebook.community/piskvorky/gensim/docs/notebooks/nmf_tutorial 

In [2]:
import os
import math
import csv
import sys
import numpy as np
import pandas as pd
import json
import glob

import statistics as stats

from sklearn import metrics
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation
from sklearn.cluster import KMeans

import gensim

import pyLDAvis
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()

from nltk.sentiment import SentimentIntensityAnalyzer
from nltk.corpus import stopwords
import operator

/Users/anikanissen/opt/anaconda3/envs/JiraSupportTickets/lib/python3.10/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib and slated for removal in Python 3.12; see the module's documentation for alternative uses
  from imp import reload


## 0. Data Preparation
Let's first load in the Excel-File with all the Jira Tickets from 2019 to 17th October 2022. Then we will derive subsets of the data and save them as their own dataframes and then go and analyze them ;)

In [3]:
#Import data and have a look at data
reports = pd.read_excel('Jira Support_Export_clean.xlsx')
reports.head()

,Vorgangstyp,Schlüssel,Zusammenfassung,Beschreibung,Zugewiesene Person,Autor,Priorität,Status,Erstellt,Aktualisiert
0,Support,SWELIB-2158,Nicht mehr im Kurs,"Hallo,\n\n\n\nich hatte den TS410 Kurs erfolgr...",Mareen Wienand,maseh_h@hotmail.de,High,Waiting for support,2022-10-16 23:16:05.364,2022-10-16 23:16:05.364
1,Support,SWELIB-2157,Purchase-To-Pay | Buch wird nicht als Abgeschl...,"Hallo zusammen,\n\nIm Bereich “Purchase-To-Pay...",David Rumpf,fabian.kaudel@outlook.de,Medium,Geschlossen,2022-10-13 10:01:19.615,2022-10-15 13:49:51.413
2,Support,SWELIB-2156,Fallstudie 1: Übung 17 - Zahlungsausgang buche...,"Hallo zusammen,\n\n\n\nich bearbeite aktuell Ü...",David Rumpf,fabian.kaudel@outlook.de,Medium,Waiting for customer,2022-10-13 09:08:35.759,2022-10-16 16:37:40.070
3,Support,SWELIB-2155,Anmeldung zur Zertifikatsprüfung,"Guten Tag,\n\nich habe im Dezember letzten Jah...",David Rumpf,Andre.statnik@hotmail.de,High,Waiting for customer,2022-10-13 00:50:06.988,2022-10-13 21:37:35.281
4,Support,SWELIB-2154,Falsche EInheit hinterlegt in Aufgabe9 der Fal...,"Hallo zusammen,\n\n \n\naktuell hänge ich an A...",Kerem Yildirim,fabian.kaudel@outlook.de,Medium,Waiting for customer,2022-10-10 19:33:46.327,2022-10-11 17:00:52.151


In [4]:
reports.describe()

/var/folders/6v/6zzptl6s5dz4ttbj13tg4vhc0000gn/T/ipykernel_30881/568569428.py:1: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  reports.describe()
/var/folders/6v/6zzptl6s5dz4ttbj13tg4vhc0000gn/T/ipykernel_30881/568569428.py:1: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  reports.describe()


,Vorgangstyp,Schlüssel,Zusammenfassung,Beschreibung,Zugewiesene Person,Autor,Priorität,Status,Erstellt,Aktualisiert
count,2068,2068,2068,1791,2046,2065,2068,2068,2068,2068
unique,1,2068,1809,1775,9,494,5,4,2068,2068
top,Support,SWELIB-2158,Neue Nachricht von es4students - Kontaktformular,TEST,WeLearninBits,mail@es4students.de,Medium,Waiting for customer,2022-10-16 23:16:05.364000,2022-10-16 23:16:05.364000
freq,2068,1,38,9,770,55,1316,1148,1,1
first,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-05-02 12:36:28.235000,2019-07-29 15:23:31.660000
last,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-10-16 23:16:05.364000,2022-10-16 23:16:05.364000


In [5]:
reports.dtypes

Vorgangstyp                   object
Schlüssel                     object
Zusammenfassung               object
Beschreibung                  object
Zugewiesene Person            object
Autor                         object
Priorität                     object
Status                        object
Erstellt              datetime64[ns]
Aktualisiert          datetime64[ns]
dtype: object

In [6]:
zsmfassungNEU = reports[reports['Erstellt'] > '2021-05-01']
zsmfassungALT = reports[reports['Erstellt'] < '2021-05-01']
zsmfassungNEU

,Vorgangstyp,Schlüssel,Zusammenfassung,Beschreibung,Zugewiesene Person,Autor,Priorität,Status,Erstellt,Aktualisiert
0,Support,SWELIB-2158,Nicht mehr im Kurs,"Hallo,\n\n\n\nich hatte den TS410 Kurs erfolgr...",Mareen Wienand,maseh_h@hotmail.de,High,Waiting for support,2022-10-16 23:16:05.364,2022-10-16 23:16:05.364
1,Support,SWELIB-2157,Purchase-To-Pay | Buch wird nicht als Abgeschl...,"Hallo zusammen,\n\nIm Bereich “Purchase-To-Pay...",David Rumpf,fabian.kaudel@outlook.de,Medium,Geschlossen,2022-10-13 10:01:19.615,2022-10-15 13:49:51.413
2,Support,SWELIB-2156,Fallstudie 1: Übung 17 - Zahlungsausgang buche...,"Hallo zusammen,\n\n\n\nich bearbeite aktuell Ü...",David Rumpf,fabian.kaudel@outlook.de,Medium,Waiting for customer,2022-10-13 09:08:35.759,2022-10-16 16:37:40.070
3,Support,SWELIB-2155,Anmeldung zur Zertifikatsprüfung,"Guten Tag,\n\nich habe im Dezember letzten Jah...",David Rumpf,Andre.statnik@hotmail.de,High,Waiting for customer,2022-10-13 00:50:06.988,2022-10-13 21:37:35.281
4,Support,SWELIB-2154,Falsche EInheit hinterlegt in Aufgabe9 der Fal...,"Hallo zusammen,\n\n \n\naktuell hänge ich an A...",Kerem Yildirim,fabian.kaudel@outlook.de,Medium,Waiting for customer,2022-10-10 19:33:46.327,2022-10-11 17:00:52.151
...,...,...,...,...,...,...,...,...,...,...
569,Support,SWELIB-1582,Neue ID-Daten bei SAP Fiori,"Sehr geehrte Damen und Herren, \n\nich habe wi...",WeLearninBits,Edou Ndong,High,Waiting for customer,2021-05-02 18:25:51.249,2021-05-03 10:01:34.836
570,Support,SWELIB-1581,Login in das neue Portal,Der Name Nitharsan Jeyam wurde meine Emailadre...,WeLearninBits,Nirushanan Srimurugan,High,Waiting for customer,2021-05-01 12:51:40.239,2021-05-03 09:47:21.555
571,Support,SWELIB-1580,Anmeldung neue Plattform,Gut Tag. Ich denke bei der Migration auf die n...,WeLearninBits,manuel.klemt@stud.uni-due.de,High,Waiting for customer,2021-05-01 11:40:59.491,2021-05-03 09:47:11.418
572,Support,SWELIB-1579,Anmeldung auf der neuen Webseite und Lernplatt...,Mit welchen Anmeldedaten kann ich mich auf der...,WeLearninBits,annika.huebner@stud.uni-due.de,High,Waiting for customer,2021-05-01 10:53:42.686,2021-05-03 09:48:52.302


In [7]:
dfALT = zsmfassungALT['Zusammenfassung']
dfALT.fillna("",inplace=True)
dfALT

/var/folders/6v/6zzptl6s5dz4ttbj13tg4vhc0000gn/T/ipykernel_30881/3749423428.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfALT.fillna("",inplace=True)


574     Guten Tag, ich habe eine Frage und zwar habe i...
575     Hallo, ich möchte gerne an der SAP Prüfung tei...
576                   Dauer der Korrektur der Fallstudien
577         Fehlermeldung Auftrag wird gerade bearbeitet 
578     Hallo, ich bin bei der Zertifizierungsprüfung ...
                              ...                        
2063        Abweichende Rechnungs- und Zustellungsadresse
2064                           Probleme bei der Anmeldung
2065                      Re: Willkommen bei es/4students
2066                                             bachelor
2067                                            Fragen...
Name: Zusammenfassung, Length: 1494, dtype: object

In [8]:
dfNeu = zsmfassungNEU['Zusammenfassung']
dfNeu.fillna("",inplace=True)
dfNeu

/var/folders/6v/6zzptl6s5dz4ttbj13tg4vhc0000gn/T/ipykernel_30881/852018920.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfNeu.fillna("",inplace=True)


0                                     Nicht mehr im Kurs
1      Purchase-To-Pay | Buch wird nicht als Abgeschl...
2      Fallstudie 1: Übung 17 - Zahlungsausgang buche...
3                       Anmeldung zur Zertifikatsprüfung
4      Falsche EInheit hinterlegt in Aufgabe9 der Fal...
                             ...                        
569                          Neue ID-Daten bei SAP Fiori
570                             Login in das neue Portal
571                            Anmeldung neue Plattform 
572    Anmeldung auf der neuen Webseite und Lernplatt...
573    I have registered for SAP TS410 with fees for ...
Name: Zusammenfassung, Length: 574, dtype: object

In [9]:
#reports['Abstract']= reports['Abstract'].astype(str)
zsmfassung = reports['Zusammenfassung']
zsmfassung.fillna("",inplace=True)

beschreibung = reports['Beschreibung']
beschreibung.fillna("",inplace=True)

## 1. Clustering the Topics for ALL Tickets

The following lines leverage these packages:
* `from sklearn.feature_extraction.text import TfidfVectorizer`
* `from sklearn.cluster import KMeans`

### 1.1. Cluster based on Zusammenfassung des Tickets


In [10]:
german_stop_words = stopwords.words('german')

tfidf = TfidfVectorizer(max_features=20000, stop_words=german_stop_words)
X = tfidf.fit_transform(zsmfassung)

clustered = KMeans(n_clusters=6, random_state=0).fit_predict(X)

#with_clusters=pd.DataFrame(clusters)
reports['cluster_zsmfassung']=clustered

grouped=reports.groupby('cluster_zsmfassung')
for gp_name, gp in grouped:
    display(gp)

grouped.describe()

,Vorgangstyp,Schlüssel,Zusammenfassung,Beschreibung,Zugewiesene Person,Autor,Priorität,Status,Erstellt,Aktualisiert,cluster_zsmfassung
1,Support,SWELIB-2157,Purchase-To-Pay | Buch wird nicht als Abgeschl...,"Hallo zusammen,\n\nIm Bereich “Purchase-To-Pay...",David Rumpf,fabian.kaudel@outlook.de,Medium,Geschlossen,2022-10-13 10:01:19.615,2022-10-15 13:49:51.413,0
2,Support,SWELIB-2156,Fallstudie 1: Übung 17 - Zahlungsausgang buche...,"Hallo zusammen,\n\n\n\nich bearbeite aktuell Ü...",David Rumpf,fabian.kaudel@outlook.de,Medium,Waiting for customer,2022-10-13 09:08:35.759,2022-10-16 16:37:40.070,0
4,Support,SWELIB-2154,Falsche EInheit hinterlegt in Aufgabe9 der Fal...,"Hallo zusammen,\n\n \n\naktuell hänge ich an A...",Kerem Yildirim,fabian.kaudel@outlook.de,Medium,Waiting for customer,2022-10-10 19:33:46.327,2022-10-11 17:00:52.151,0
5,Support,SWELIB-2153,Alle Transaktionen mit LEARN-192 löschen,"Guten Tag,\n\n\n\nich möchte die Übungen nochm...",David Rumpf,zdravka.todorova80@gmail.com,Medium,Waiting for customer,2022-10-07 08:24:35.460,2022-10-07 17:16:56.698,0
6,Support,SWELIB-2152,Server ist unerreichbar,"Guten Tag,\n\nder Server ist seit zwei drei Ta...",David Rumpf,zdravka.todorova80@gmail.com,Medium,Geschlossen,2022-10-06 19:22:55.959,2022-10-07 17:08:39.064,0
...,...,...,...,...,...,...,...,...,...,...,...
2062,Support,SWELIB-26,Passwort zurücksetzen,Zur Info: die Anfrage wurde in unser Ticketsys...,Ehemaliger Benutzer,movin4mind@gmail.com,High,Geschlossen,2019-05-09 10:17:47.494,2019-09-07 10:00:39.901,0
2063,Support,SWELIB-24,Abweichende Rechnungs- und Zustellungsadresse,"Sehr geehrte Damen und Herren,\n\nich würde ge...",Ehemaliger Benutzer,alexandraklein.x3@gmail.com,High,Geschlossen,2019-05-07 13:40:01.723,2019-09-07 10:00:37.750,0
2065,Support,SWELIB-20,Re: Willkommen bei es/4students,"Sehr geehrte Damen und Herren,\n\nich habe mic...",NaN,peymud@gmail.com,High,Geschlossen,2019-05-05 13:41:50.328,2019-09-07 10:00:37.428,0
2066,Support,SWELIB-19,bachelor,ich finde nicht eine Sache in GUI Mac und desh...,NaN,liuyangtest123@gmail.com,Medium,Geschlossen,2019-05-02 15:33:17.180,2019-09-07 10:00:37.507,0


,Vorgangstyp,Schlüssel,Zusammenfassung,Beschreibung,Zugewiesene Person,Autor,Priorität,Status,Erstellt,Aktualisiert,cluster_zsmfassung
19,Support,SWELIB-2139,"Guten Tag,",,David Rumpf,zdravka.todorova80@gmail.com,Medium,Waiting for customer,2022-09-24 15:37:43.464,2022-10-01 18:32:15.081,1
102,Support,SWELIB-2056,"Guten Tag, ich würde die SAP-Prüfung gerne zei...",,David Rumpf,Hendrik Wannmacher,Medium,Geschlossen,2022-06-30 14:10:37.220,2022-10-02 11:10:55.206,1
308,Support,SWELIB-1849,"Guten Tag, ich wollte mich einmal erkundigen, ...",,WeLearninBits,oliver1515@gmx.de,Medium,Waiting for customer,2021-10-12 11:59:32.196,2021-10-14 11:23:34.175,1
316,Support,SWELIB-1841,"Guten Tag, ich möchte gerne einen Prüfungsterm...",,WeLearninBits,Erva Cinar,Medium,Waiting for customer,2021-10-05 12:01:44.651,2021-10-08 14:30:29.933,1
399,Support,SWELIB-1755,"Guten Tag, ich wollte fragen ob ich einen weit...",,WeLearninBits,Wilhelm Wenz,Medium,Waiting for customer,2021-09-02 13:38:40.559,2021-09-03 11:52:38.183,1
416,Support,SWELIB-1736,"Guten Tag, ich habe letzten Freitag (20.08.202...",,WeLearninBits,Dominik Schimanski,Medium,Waiting for customer,2021-08-27 13:39:07.112,2021-08-30 11:22:38.042,1
452,Support,SWELIB-1700,"Guten Tag Herr Schott, leider steht auf die SA...",,WeLearninBits,koblavimarcarnaud@hotmail.com,Medium,Waiting for customer,2021-08-05 12:47:57.497,2021-08-16 21:04:54.874,1
455,Support,SWELIB-1697,"Guten Tag, ich würde gerne wissen wie lange es...",,WeLearninBits,koblavimarcarnaud@hotmail.com,Medium,Geschlossen,2021-08-02 15:02:30.897,2021-08-05 11:21:33.477,1
456,Support,SWELIB-1696,"Guten Tag,SAP TS410 - Bescheinigung der Univer...",,Mareen Wienand,koblavimarcarnaud@hotmail.com,Medium,Geschlossen,2021-08-02 14:58:56.487,2021-08-03 10:55:15.925,1
513,Support,SWELIB-1638,Guten Tag und zwar steht bei mir in der Prüfun...,,WeLearninBits,Wilhelm Wenz,Medium,Waiting for customer,2021-06-11 13:40:56.154,2021-06-14 10:54:18.969,1


,Vorgangstyp,Schlüssel,Zusammenfassung,Beschreibung,Zugewiesene Person,Autor,Priorität,Status,Erstellt,Aktualisiert,cluster_zsmfassung
104,Support,SWELIB-2054,"Hallo, ich bin heute durch die SAP Prüfung dur...",,Mareen Wienand,Beata Broda,Medium,Waiting for customer,2022-06-26 12:47:50.241,2022-07-17 19:05:19.035,2
107,Support,SWELIB-2050,Kennwort für SAP Fiori lässt sich nicht zurück...,"Guten Tag,\n\n\n\nich habe versucht mein Passw...",Kerem Yildirim,zdravka.todorova80@gmail.com,High,Waiting for customer,2022-06-20 19:03:01.789,2022-08-30 15:24:41.296,2
108,Support,SWELIB-2049,SAP Problem,Bei der Fallstudie 2-Übung 4: Anfrage anlegen ...,Mareen Wienand,honghao.van10@gmail.com,Medium,Waiting for customer,2022-06-17 14:42:48.324,2022-06-20 11:32:44.527,2
114,Support,SWELIB-2043,SAP Success Factors,"Hallo, \n\nwird in absehbarer Zeit der Kurs SA...",Mareen Wienand,tiyes@hotmail.de,High,Waiting for customer,2022-06-11 11:13:14.351,2022-06-11 11:35:47.048,2
124,Support,SWELIB-2033,Kann mich in SAP Fiori nicht anmelden,Ich habe am 18.05 die Zugangsdaten erhalten ab...,Kerem Yildirim,aphisit.singsiri@web.de,High,Geschlossen,2022-05-23 17:04:49.111,2022-05-27 16:51:58.057,2
...,...,...,...,...,...,...,...,...,...,...,...
2024,Support,SWELIB-69,SAP GUI für MAC,Bitte senden Sie mir eine analoge Beschreibung...,Ehemaliger Benutzer,steffen.schroeder5@freenet.de,Medium,Geschlossen,2019-06-07 06:53:55.141,2019-09-07 10:00:40.098,2
2028,Support,SWELIB-65,AW: ES-56 SAP Fiori Oberfläche gesperrt,"Hallo Felix, \n\n \n\nkönntest du mir den Sta...",Ehemaliger Benutzer,leonie.reymann@uni-due.de,High,Geschlossen,2019-06-04 09:15:26.999,2019-09-07 10:00:37.721,2
2032,Support,SWELIB-61,AW: ES-56 SAP Fiori Oberfläche gesperrt,{color:#333333} {color}{color:#000000} \n\nHal...,Ehemaliger Benutzer,leonie.reymann@uni-due.de,High,Geschlossen,2019-06-03 11:43:40.671,2019-09-07 10:00:37.790,2
2033,Support,SWELIB-60,SAP GUI Installation,"Hallo, \n\nich bin habe mir die Einleitung sow...",Dustin Syfuß,Moira.regen@stud.uni-due.de,Medium,Geschlossen,2019-06-02 17:31:23.659,2019-09-07 10:00:40.136,2


,Vorgangstyp,Schlüssel,Zusammenfassung,Beschreibung,Zugewiesene Person,Autor,Priorität,Status,Erstellt,Aktualisiert,cluster_zsmfassung
3,Support,SWELIB-2155,Anmeldung zur Zertifikatsprüfung,"Guten Tag,\n\nich habe im Dezember letzten Jah...",David Rumpf,Andre.statnik@hotmail.de,High,Waiting for customer,2022-10-13 00:50:06.988,2022-10-13 21:37:35.281,3
68,Support,SWELIB-2090,Anmeldung auf Moodle nicht möglich,"Guten Tag, \n\nich melde mich nun schon zum zw...",David Rumpf,Jannis Vedder,High,Geschlossen,2022-08-10 19:53:39.371,2022-10-02 11:11:01.859,3
69,Support,SWELIB-2089,Moodle Anmeldung funktioniert nicht,"Guten Tag,\n\n ich habe das Problem, dass ich ...",David Rumpf,Jannis Vedder,High,Geschlossen,2022-08-09 11:16:07.178,2022-08-21 16:58:06.099,3
77,Support,SWELIB-2081,Anmeldung auf der neuen Platform,"Sehr geehrte Damen und Herren, \n\nich habe nu...",Mareen Wienand,tobilo2@web.de,High,Waiting for customer,2022-08-03 10:04:36.793,2022-08-03 11:38:07.725,3
80,Support,SWELIB-2078,Anmeldung auf der Lernplattform nicht möglich,"Hallo,\n\nIch kann mich nicht auf der neuen Le...",Mareen Wienand,annaunkel@gmx.de,High,Waiting for customer,2022-07-28 10:56:45.675,2022-07-28 11:06:11.550,3
...,...,...,...,...,...,...,...,...,...,...,...
1991,Support,SWELIB-106,SAP GUI Anmeldung nicht möglich.,"Hallo,\nbeim erstmaligen Anmeldeversuch in SAP...",Daniel Pleuger,martin.musser@outlook.com,High,Geschlossen,2019-06-30 14:57:46.555,2019-09-07 10:00:37.627,3
2009,Support,SWELIB-88,Anmeldung funktioniert nicht,"Ich habe mich für den SAP-Kurs ""es/4students -...",Ehemaliger Benutzer,christian.rohde1993@googlemail.com,High,Geschlossen,2019-06-17 19:39:38.189,2019-09-07 10:00:38.412,3
2030,Support,SWELIB-63,Anmeldung bei SAP Logon,"Sehr geehrtes Support-Team,\n\nich bin wie in ...",Dustin Syfuß,Daniel Heinzig,Medium,Geschlossen,2019-06-03 16:51:03.290,2019-09-07 10:00:40.176,3
2058,Support,SWELIB-30,Wie geht es nach der Anmeldung weiter?,"Hallo,\n\nich habe mich gestern angemeldet und...",Ehemaliger Benutzer,Cagatay Demirtas,High,Geschlossen,2019-05-09 14:43:28.412,2019-09-07 10:00:39.194,3


,Vorgangstyp,Schlüssel,Zusammenfassung,Beschreibung,Zugewiesene Person,Autor,Priorität,Status,Erstellt,Aktualisiert,cluster_zsmfassung
346,Support,SWELIB-1811,New Message From es4students - Kontaktformular,%%Name%%_x000D_\n%%Email%%_x000D_\n%%Message%%...,Mareen Wienand,mail@es4students.com,High,Geschlossen,2021-09-17 02:17:42.748,2021-09-17 17:19:24.761,4
390,Support,SWELIB-1767,New Message From es4students - Kontaktformular,%%Name%%_x000D_\n%%Email%%_x000D_\n%%Message%%...,Mareen Wienand,mail@es4students.com,High,Geschlossen,2021-09-04 04:22:56.758,2021-09-06 11:24:53.833,4
395,Support,SWELIB-1760,New Message From es4students - Kontaktformular,%%Name%%_x000D_\n%%Email%%_x000D_\n%%Message%%...,Mareen Wienand,mail@es4students.com,High,Geschlossen,2021-09-02 22:39:30.964,2021-09-03 09:52:39.734,4
414,Support,SWELIB-1738,New Message From es4students - Kontaktformular,%%Name%%_x000D_\n%%Email%%_x000D_\n%%Message%%...,Mareen Wienand,mail@es4students.com,High,Geschlossen,2021-08-29 02:32:08.862,2021-08-30 13:52:49.922,4
475,Support,SWELIB-1677,New Message From es4students - Kontaktformular,%%Name%%_x000D_\n%%Email%%_x000D_\n%%Message%%...,Mareen Wienand,mail@es4students.com,High,Geschlossen,2021-07-15 01:43:24.993,2021-07-16 11:02:56.076,4
...,...,...,...,...,...,...,...,...,...,...,...
2036,Support,SWELIB-57,New Message From es4students - Kontaktieren Si...,Mark Leifeld_x000D_\nmarkleifeld.home@live.com...,Ehemaliger Benutzer,mail@es4students.de,High,Geschlossen,2019-05-31 11:32:46.584,2019-09-07 10:00:38.387,4
2039,Support,SWELIB-52,New Message From es4students - Kontaktieren Si...,Tobias Mohner_x000D_\ntobias.mohner@studium.fe...,Dustin Syfuß,mail@es4students.de,High,Geschlossen,2019-05-29 09:55:37.877,2019-09-07 10:00:40.190,4
2042,Support,SWELIB-49,New Message From es4students - Kontaktieren Si...,Nafiseh Ahmadi_x000D_\nNafiseh.ahmadi@hotmail....,Dustin Syfuß,mail@es4students.de,High,Geschlossen,2019-05-23 17:16:06.492,2019-09-07 10:00:39.749,4
2048,Support,SWELIB-43,New Message From es4students - Kontaktieren Si...,Niklas Lenk_x000D_\nniklas.l@unitybox.de_x000D...,Dustin Syfuß,NaN,High,Geschlossen,2019-05-20 23:20:13.979,2019-09-07 10:00:39.578,4


,Vorgangstyp,Schlüssel,Zusammenfassung,Beschreibung,Zugewiesene Person,Autor,Priorität,Status,Erstellt,Aktualisiert,cluster_zsmfassung
0,Support,SWELIB-2158,Nicht mehr im Kurs,"Hallo,\n\n\n\nich hatte den TS410 Kurs erfolgr...",Mareen Wienand,maseh_h@hotmail.de,High,Waiting for support,2022-10-16 23:16:05.364,2022-10-16 23:16:05.364,5
8,Support,SWELIB-2150,Der SAP Kurs wird mir trotz erfolgreicher Best...,,Kerem Yildirim,v.schulzedieckhoff@gmail.com,High,Waiting for customer,2022-10-06 00:54:27.453,2022-10-06 11:40:19.636,5
16,Support,SWELIB-2142,Neuerwerbung Kurs,"Sehr geehrte Damen und Herren, \n\nich habe de...",Kerem Yildirim,andreberghoff@gmx.de,High,Waiting for customer,2022-09-27 13:29:24.074,2022-09-28 15:00:48.460,5
56,Support,SWELIB-2102,Rechnung für TS410 Kurs,"Guten Tag,\n\n\n\nwäre es möglich, dass Sie mi...",Mareen Wienand,Robin Helbig,High,Waiting for customer,2022-08-24 14:47:06.703,2022-08-26 20:18:53.532,5
76,Support,SWELIB-2082,Rückmeldung Kurs abschließen,"Sehr geehrtes we learn in bits -Team, \nletzte...",Kerem Yildirim,simon_josef.heilmeier@fom-net.de,Medium,Waiting for customer,2022-08-04 10:18:37.376,2022-08-04 17:20:28.846,5
...,...,...,...,...,...,...,...,...,...,...,...
1510,Support,SWELIB-634,Kurs C_TS410 Kurswechsel,"Sehr geehrtes Es4students-Team,\n\nim Sommer v...",Ehemaliger Benutzer,steinberg.allegra@gmail.com,High,Geschlossen,2020-01-16 20:03:01.222,2021-11-16 12:36:49.769,5
1513,Support,SWELIB-631,Bestätigung Abschluss SAP Kurs,"Hallo, ich habe am 12.12. die Mail geschickt, ...",Ehemaliger Benutzer,JuKoe98@web.de,High,Geschlossen,2020-01-08 15:56:26.564,2020-01-30 13:32:44.425,5
1533,Support,SWELIB-611,BOWI-Kurs,"Sehr geehrte Damen und Herren,\nich mache seit...",Ehemaliger Benutzer,kankoe.chardey@stud.uni-due.de,High,Geschlossen,2019-12-27 14:53:22.211,2021-11-16 12:34:52.293,5
1566,Support,SWELIB-578,SAP-Kurs Verlängerung durch Wartrungsarbeiten,"Sehr geehrtes Team von es4students,\n\nbis wan...",Ehemaliger Benutzer,Nikola Mirkovic,High,Geschlossen,2019-12-10 13:27:30.761,2020-01-30 12:12:54.107,5


/var/folders/6v/6zzptl6s5dz4ttbj13tg4vhc0000gn/T/ipykernel_30881/202582502.py:15: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  grouped.describe()
/var/folders/6v/6zzptl6s5dz4ttbj13tg4vhc0000gn/T/ipykernel_30881/202582502.py:15: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  grouped.describe()
/var/folders/6v/6zzptl6s5dz4ttbj13tg4vhc0000gn/T/ipykernel_30881/202582502.py:15: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and ad

Vorgangstyp                                  Schlüssel  \
                         count unique      top  freq first last     count   
cluster_zsmfassung                                                          
0                         1541      1  Support  1541   NaN  NaN      1541   
1                           37      1  Support    37   NaN  NaN        37   
2                          244      1  Support   244   NaN  NaN       244   
3                          108      1  Support   108   NaN  NaN       108   
4                           76      1  Support    76   NaN  NaN        76   
5                           62      1  Support    62   NaN  NaN        62   

                                             ...                    Erstellt  \
                   unique          top freq  ...                         top   
cluster_zsmfassung                           ...                               
0                    1541  SWELIB-2157    1  ...  2022-10-13 10:01:19.615000   
1                      37  SWELIB-2139    1  ...  2022-09-24 15:37:43.464000   
2                     244  SWELIB-2054    1  ...  2022-06-26 12:47:50.241000   
3                     108  SWELIB-2155    1  ...  2022-10-13 00:50:06.988000   
4                      76  SWELIB-1811    1  ...  2021-09-17 02:17:42.748000   
5                      62  SWELIB-2158    1  ...  2022-10-16 23:16:05.364000   

                                                     \
                   freq                       first   
cluster_zsmfassung                                    
0                     1  2019-05-02 12:36:28.235000   
1                     1  2019-09-07 19:34:36.170000   
2                     1  2019-05-30 12:49:54.589000   
3                     1  2019-05-06 11:57:46.765000   
4                     1  2019-05-09 11:48:48.584000   
5                     1  2019-07-31 11:24:33.850000   

                                               Aktualisiert         \
                                          last        count unique   
cluster_zsmfassung                                                   
0                   2022-10-13 10:01:19.615000         1541   1541   
1                   2022-09-24 15:37:43.464000           37     37   
2                   2022-06-26 12:47:50.241000          244    244   
3                   2022-10-13 00:50:06.988000          108    108   
4                   2021-09-17 02:17:42.748000           76     76   
5                   2022-10-16 23:16:05.364000           62     62   

                                                     \
                                           top freq   
cluster_zsmfassung                                    
0                   2022-10-15 13:49:51.413000    1   
1                   2022-10-01 18:32:15.081000    1   
2                   2022-07-17 19:05:19.035000    1   
3                   2022-10-13 21:37:35.281000    1   
4                   2021-09-17 17:19:24.761000    1   
5                   2022-10-16 23:16:05.364000    1   

                                                                            
                                         first                        last  
cluster_zsmfassung                                                          
0                   2019-07-30 12:38:58.894000  2022-10-16 16:37:40.070000  
1                   2019-09-10 09:53:16.285000  2022-10-02 11:10:55.206000  
2                   2019-07-29 15:32:12.936000  2022-08-30 15:24:41.296000  
3                   2019-08-03 11:56:47.606000  2022-10-13 21:37:35.281000  
4                   2019-07-29 15:23:31.660000  2021-09-17 17:19:24.761000  
5                   2019-08-04 11:39:53.426000  2022-10-16 23:16:05.364000  

[6 rows x 60 columns]

### 1.2. Cluster based on Beschreibung des Tickets

In [11]:
tfidf = TfidfVectorizer(max_features=20000, stop_words=german_stop_words)
X = tfidf.fit_transform(beschreibung)

clustered = KMeans(n_clusters=6, random_state=0).fit_predict(X)

#with_clusters=pd.DataFrame(clusters)
reports['cluster_beschreibung']=clustered

grouped_beschreibung=reports.groupby('cluster_beschreibung')
for gp_name, gp in grouped_beschreibung:
    display(gp)

grouped_beschreibung.describe()

,Vorgangstyp,Schlüssel,Zusammenfassung,Beschreibung,Zugewiesene Person,Autor,Priorität,Status,Erstellt,Aktualisiert,cluster_zsmfassung,cluster_beschreibung
6,Support,SWELIB-2152,Server ist unerreichbar,"Guten Tag,\n\nder Server ist seit zwei drei Ta...",David Rumpf,zdravka.todorova80@gmail.com,Medium,Geschlossen,2022-10-06 19:22:55.959,2022-10-07 17:08:39.064,0,0
7,Support,SWELIB-2151,Moodle geht nicht,"Hallo,\n\nvor zwei Tagen konnte man sich noch ...",David Rumpf,Jannis Vedder,High,Geschlossen,2022-10-06 16:22:14.098,2022-10-07 17:04:13.564,0,0
10,Support,SWELIB-2148,Server ist unerreichbar,"Guten Tag,\n\nseit einigen STunden versuche ic...",David Rumpf,zdravka.todorova80@gmail.com,Medium,Waiting for customer,2022-10-01 14:16:29.446,2022-10-01 22:58:52.898,0,0
21,Support,SWELIB-2137,Zugriff auf Lernplattform,"Guten Tag,\n\nich wollte mit dem Kurs starten,...",David Rumpf,Jakob Zinnitsch,High,In Arbeit,2022-09-19 12:21:44.007,2022-09-22 10:59:03.573,0,0
29,Support,SWELIB-2129,Log on on SAP FIORI not possible,"Hallo,\n\nich kann mich leider immer noch nich...",Kerem Yildirim,dave.a.hanna@hotmail.com,Medium,Waiting for customer,2022-09-12 14:19:58.149,2022-09-12 15:09:20.901,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
2047,Support,SWELIB-44,Login funktioniert nicht mehr,Am 19.05. habe ich mich für das SAP Beraterzer...,Dustin Syfuß,steinberg.allegra@gmail.com,High,Geschlossen,2019-05-21 08:49:27.825,2019-09-07 10:00:37.805,0,0
2057,Support,SWELIB-32,"Eintragen von ""zugesandten Daten""","Guten Tag,\nBei der Einrichtung von SAP Logon ...",Dustin Syfuß,denisim2009@hotmail.de,Medium,Geschlossen,2019-05-12 16:54:01.196,2019-09-07 10:00:37.818,0,0
2060,Support,SWELIB-28,Zurücksetzten Passwort,Überführt in Supportsystem:\n\nMartin Musser <...,Ehemaliger Benutzer,martin.musser@outlook.com,High,Geschlossen,2019-05-09 11:46:04.820,2019-09-07 10:00:37.884,0,0
2062,Support,SWELIB-26,Passwort zurücksetzen,Zur Info: die Anfrage wurde in unser Ticketsys...,Ehemaliger Benutzer,movin4mind@gmail.com,High,Geschlossen,2019-05-09 10:17:47.494,2019-09-07 10:00:39.901,0,0


,Vorgangstyp,Schlüssel,Zusammenfassung,Beschreibung,Zugewiesene Person,Autor,Priorität,Status,Erstellt,Aktualisiert,cluster_zsmfassung,cluster_beschreibung
139,Support,SWELIB-2018,SAP,"Sehr geehrte Damen und Herren,_x000D_\n_x000D_...",NaN,samet.baspinar@stud.uni-due.de,High,Waiting for customer,2022-05-01 19:50:05.136,2022-05-02 19:44:12.722,2,1
335,Support,SWELIB-1822,SAP Prüfung,Sehr geehrte Damen und Herren_x000D_\n_x000D_\...,WeLearninBits,Aysenur Gönülalan,High,Waiting for customer,2021-09-23 21:17:48.319,2021-10-03 17:55:39.279,2,1
346,Support,SWELIB-1811,New Message From es4students - Kontaktformular,%%Name%%_x000D_\n%%Email%%_x000D_\n%%Message%%...,Mareen Wienand,mail@es4students.com,High,Geschlossen,2021-09-17 02:17:42.748,2021-09-17 17:19:24.761,4,1
356,Support,SWELIB-1801,Problem über SAP Portal,Sehr geehrte Damen und Herren_x000D_\n_x000D_\...,WeLearninBits,Aysenur Gönülalan,High,Waiting for customer,2021-09-14 23:37:41.403,2021-09-15 14:41:18.931,2,1
390,Support,SWELIB-1767,New Message From es4students - Kontaktformular,%%Name%%_x000D_\n%%Email%%_x000D_\n%%Message%%...,Mareen Wienand,mail@es4students.com,High,Geschlossen,2021-09-04 04:22:56.758,2021-09-06 11:24:53.833,4,1
...,...,...,...,...,...,...,...,...,...,...,...,...
2039,Support,SWELIB-52,New Message From es4students - Kontaktieren Si...,Tobias Mohner_x000D_\ntobias.mohner@studium.fe...,Dustin Syfuß,mail@es4students.de,High,Geschlossen,2019-05-29 09:55:37.877,2019-09-07 10:00:40.190,4,1
2042,Support,SWELIB-49,New Message From es4students - Kontaktieren Si...,Nafiseh Ahmadi_x000D_\nNafiseh.ahmadi@hotmail....,Dustin Syfuß,mail@es4students.de,High,Geschlossen,2019-05-23 17:16:06.492,2019-09-07 10:00:39.749,4,1
2048,Support,SWELIB-43,New Message From es4students - Kontaktieren Si...,Niklas Lenk_x000D_\nniklas.l@unitybox.de_x000D...,Dustin Syfuß,NaN,High,Geschlossen,2019-05-20 23:20:13.979,2019-09-07 10:00:39.578,4,1
2051,Support,SWELIB-38,Tom Adams - Keine Mail-Adresse,"Sehr geehrte Damen und Herren,_x000D_\n_x000D_...",Ehemaliger Benutzer,mail@es4students.de,High,Geschlossen,2019-05-19 17:55:59.605,2019-09-07 10:00:40.020,0,1


,Vorgangstyp,Schlüssel,Zusammenfassung,Beschreibung,Zugewiesene Person,Autor,Priorität,Status,Erstellt,Aktualisiert,cluster_zsmfassung,cluster_beschreibung
1,Support,SWELIB-2157,Purchase-To-Pay | Buch wird nicht als Abgeschl...,"Hallo zusammen,\n\nIm Bereich “Purchase-To-Pay...",David Rumpf,fabian.kaudel@outlook.de,Medium,Geschlossen,2022-10-13 10:01:19.615,2022-10-15 13:49:51.413,0,2
4,Support,SWELIB-2154,Falsche EInheit hinterlegt in Aufgabe9 der Fal...,"Hallo zusammen,\n\n \n\naktuell hänge ich an A...",Kerem Yildirim,fabian.kaudel@outlook.de,Medium,Waiting for customer,2022-10-10 19:33:46.327,2022-10-11 17:00:52.151,0,2
5,Support,SWELIB-2153,Alle Transaktionen mit LEARN-192 löschen,"Guten Tag,\n\n\n\nich möchte die Übungen nochm...",David Rumpf,zdravka.todorova80@gmail.com,Medium,Waiting for customer,2022-10-07 08:24:35.460,2022-10-07 17:16:56.698,0,2
8,Support,SWELIB-2150,Der SAP Kurs wird mir trotz erfolgreicher Best...,,Kerem Yildirim,v.schulzedieckhoff@gmail.com,High,Waiting for customer,2022-10-06 00:54:27.453,2022-10-06 11:40:19.636,5,2
14,Support,SWELIB-2144,Technisches Problem,"Hallo,\nich kann auf ihre sogenannten ""Bücher""...",Kerem Yildirim,ouamar.ayoub@gmx.de,Medium,Waiting for customer,2022-09-28 15:13:51.716,2022-09-29 01:16:55.271,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...
2045,Support,SWELIB-46,Übung Anlegen eines Angebots,Meine SAP Seite ist abgestürtzt ich kan meine ...,Dustin Syfuß,ranjrebuarali@outlook.de,Medium,Geschlossen,2019-05-22 15:08:29.025,2019-09-07 10:00:37.598,0,2
2049,Support,SWELIB-40,Zwischentest Plan to Produce -,Beim Zwischentest von dem Modul Plan to produc...,Dustin Syfuß,ranjrebuarali@outlook.de,High,Geschlossen,2019-05-19 20:34:49.623,2019-09-07 10:00:37.345,0,2
2052,Support,SWELIB-37,Übung Buchen eines Zahlungsausgangs,Nach dem ich alle Angabe wie von der Fallstudi...,Dustin Syfuß,ranjrebuarali@outlook.de,Medium,Geschlossen,2019-05-19 17:32:04.081,2019-09-07 10:00:37.776,0,2
2054,Support,SWELIB-35,Organisationsebenen - Vorlage,Was muss ich machen damit bei mir auf der rech...,Dustin Syfuß,manuel.klemt@stud.uni-due.de,Medium,Geschlossen,2019-05-16 19:23:34.654,2019-09-07 10:00:37.639,0,2


,Vorgangstyp,Schlüssel,Zusammenfassung,Beschreibung,Zugewiesene Person,Autor,Priorität,Status,Erstellt,Aktualisiert,cluster_zsmfassung,cluster_beschreibung
0,Support,SWELIB-2158,Nicht mehr im Kurs,"Hallo,\n\n\n\nich hatte den TS410 Kurs erfolgr...",Mareen Wienand,maseh_h@hotmail.de,High,Waiting for support,2022-10-16 23:16:05.364,2022-10-16 23:16:05.364,5,3
3,Support,SWELIB-2155,Anmeldung zur Zertifikatsprüfung,"Guten Tag,\n\nich habe im Dezember letzten Jah...",David Rumpf,Andre.statnik@hotmail.de,High,Waiting for customer,2022-10-13 00:50:06.988,2022-10-13 21:37:35.281,3,3
9,Support,SWELIB-2149,Einschreibung in die Moodle Gruppe „Vorbereitu...,"Guten Morgen,\n\nIch wurde kürzlich, ca vor 2 ...",Kerem Yildirim,ouamar.ayoub@gmx.de,Medium,Waiting for customer,2022-10-04 08:44:58.492,2022-10-04 21:16:13.278,0,3
13,Support,SWELIB-2145,"Finaler Abschluss des Kurses ""Integrierte Gesc...","Sehr geehrte Damen und Herren,\n\nleider erhal...",David Rumpf,gisa.graefe@web.de,Medium,Waiting for customer,2022-09-28 16:58:36.560,2022-09-29 11:45:55.249,0,3
16,Support,SWELIB-2142,Neuerwerbung Kurs,"Sehr geehrte Damen und Herren, \n\nich habe de...",Kerem Yildirim,andreberghoff@gmx.de,High,Waiting for customer,2022-09-27 13:29:24.074,2022-09-28 15:00:48.460,5,3
...,...,...,...,...,...,...,...,...,...,...,...,...
2059,Support,SWELIB-29,Re: New Message From es4students - Kontaktiere...,Überführt in Supportsystem:\nAmit Bajwa <amits...,Ehemaliger Benutzer,amitsinghbajwa@gmail.com,High,Geschlossen,2019-05-09 11:48:48.584,2019-09-07 10:00:39.619,4,3
2061,Support,SWELIB-27,AW: es/4students Account aktivieren,"Überführt ins Supportsystem:\n\nReymann, Leoni...",Ehemaliger Benutzer,leonie.reymann@uni-due.de,High,Geschlossen,2019-05-09 11:45:10.928,2019-09-07 10:00:37.948,0,3
2063,Support,SWELIB-24,Abweichende Rechnungs- und Zustellungsadresse,"Sehr geehrte Damen und Herren,\n\nich würde ge...",Ehemaliger Benutzer,alexandraklein.x3@gmail.com,High,Geschlossen,2019-05-07 13:40:01.723,2019-09-07 10:00:37.750,0,3
2065,Support,SWELIB-20,Re: Willkommen bei es/4students,"Sehr geehrte Damen und Herren,\n\nich habe mic...",NaN,peymud@gmail.com,High,Geschlossen,2019-05-05 13:41:50.328,2019-09-07 10:00:37.428,0,3


,Vorgangstyp,Schlüssel,Zusammenfassung,Beschreibung,Zugewiesene Person,Autor,Priorität,Status,Erstellt,Aktualisiert,cluster_zsmfassung,cluster_beschreibung
2,Support,SWELIB-2156,Fallstudie 1: Übung 17 - Zahlungsausgang buche...,"Hallo zusammen,\n\n\n\nich bearbeite aktuell Ü...",David Rumpf,fabian.kaudel@outlook.de,Medium,Waiting for customer,2022-10-13 09:08:35.759,2022-10-16 16:37:40.070,0,4
11,Support,SWELIB-2147,Fehler beim Abschließen des Kurses,"Hallo,\n\nleider kann ich den Kurs nicht absch...",David Rumpf,ouamar.ayoub@gmx.de,Medium,Waiting for customer,2022-09-30 15:58:57.974,2022-10-01 18:17:22.141,0,4
12,Support,SWELIB-2146,"Fehler, order to cash Übung 1/Fallstudie2",Schritt 10: sollte Verkaufsmaterial nach Besch...,David Rumpf,zdravka.todorova80@gmail.com,Medium,Waiting for customer,2022-09-30 10:38:42.699,2022-10-01 18:23:32.720,0,4
15,Support,SWELIB-2143,Fehler,Fallstudie 2 in Purhcase to pay startet mit be...,Mareen Wienand,zdravka.todorova80@gmail.com,Medium,Waiting for customer,2022-09-28 08:49:45.934,2022-09-28 08:55:05.771,0,4
17,Support,SWELIB-2141,Zweite Übung der ersten Fallstudie nicht durch...,"Hallo zusammen,\n\nleider kann ich keine “Ange...",Kerem Yildirim,fabian.kaudel@outlook.de,Medium,Waiting for customer,2022-09-26 18:25:14.482,2022-09-30 15:09:05.776,0,4
...,...,...,...,...,...,...,...,...,...,...,...,...
2040,Support,SWELIB-51,Nummer wird nicht angezeigt,"Hallo,\nich habe die Übung ""Anlegen und Buchen...",Ehemaliger Benutzer,Cagatay Demirtas,Medium,Geschlossen,2019-05-25 01:44:06.858,2019-09-07 10:00:39.682,0,4
2041,Support,SWELIB-50,Nummer wurde nicht generiert,"Hallo,\nin dieser Aufgabe habe ich zweimal auf...",Dustin Syfuß,Cagatay Demirtas,Medium,Geschlossen,2019-05-23 18:11:51.986,2019-09-07 10:00:38.182,0,4
2050,Support,SWELIB-39,Schritt – Buchen Warenausgang zum Fertigungsau...,Ich habe die Warenausgangsbuchung machen wolle...,Dustin Syfuß,ranjrebuarali@outlook.de,Medium,Geschlossen,2019-05-19 18:15:16.619,2019-09-07 10:00:37.585,0,4
2055,Support,SWELIB-34,Meine 3-stellige Nummer,"Guten Tag,\nIn der ersten Übung bei der P2P Fa...",Dustin Syfuß,denisim2009@hotmail.de,Medium,Geschlossen,2019-05-13 14:14:36.037,2019-09-07 10:00:37.373,0,4


,Vorgangstyp,Schlüssel,Zusammenfassung,Beschreibung,Zugewiesene Person,Autor,Priorität,Status,Erstellt,Aktualisiert,cluster_zsmfassung,cluster_beschreibung
63,Support,SWELIB-2095,Some Problems and Questions - TS410EN -,"{color:#000000}Dear WeLearninBits Team,{color}...",David Rumpf,bassaksenturk@outlook.com,High,Geschlossen,2022-08-15 01:30:23.705,2022-10-02 11:10:07.965,0,5
89,Support,SWELIB-2069,Subject: Re: SWELIB-2066 Von: (LEARN- 110)--Pr...,"{color:#500050}Hallo Team Es4,{color} \n{color...",Kerem Yildirim,nagasirishanaidu3@gmail.com,High,Waiting for customer,2022-07-20 08:04:38.703,2022-07-20 11:30:49.866,0,5
92,Support,SWELIB-2066,Von: (LEARN- 110)--Problem mit Vertrieb prozes...,{color:#313131}Hallo Team{color} {color:#31313...,Kerem Yildirim,nagasirishan94@gmail.com,High,Geschlossen,2022-07-19 08:17:48.932,2022-07-20 11:32:54.331,0,5
125,Support,SWELIB-2032,Re: SWELIB-1924 GBI-358 (LEARN-110),"{color:#500050}Hallo Team Es4,{color} \n\n{col...",Kerem Yildirim,nagasirishanaidu3@gmail.com,High,Geschlossen,2022-05-23 16:24:00.974,2022-05-25 11:51:53.428,0,5
128,Support,SWELIB-2029,SWELIB-1924 GBI-358 (LEARN-110),{color:#313131}Hallo Team{color} {color:#31313...,Kerem Yildirim,nagasirishanaidu3@gmail.com,High,Geschlossen,2022-05-16 11:00:35.805,2022-05-23 15:44:35.657,0,5
186,Support,SWELIB-1971,Re: SWELIB-1968 Fehler beim Verkaufsbeleg und ...,"Hallo Herr Yildirim,\n\ndanke für die schnelle...",Mareen Wienand,ucarfatih38@googlemail.com,High,Geschlossen,2022-03-16 16:45:34.667,2022-03-18 16:57:23.585,0,5
228,Support,SWELIB-1929,Re: SWELIB-1927 Fehler bei Rechnung für Mietau...,"Hallo Herr Yildirim, \n\n \n\nleider funktion...",Kerem Yildirim,tristan.zielinski@googlemail.com,High,Waiting for customer,2022-01-16 14:38:40.843,2022-01-17 17:45:46.319,0,5
428,Support,SWELIB-1724,Re: SWELIB-1715 Anmeldung zu SAP Fiori funktio...,"Und vom Handy, habe ich folgende Probleme scho...",WeLearninBits,duygukaranfil@me.com,High,Geschlossen,2021-08-24 13:07:11.185,2021-08-24 16:20:58.563,3,5
429,Support,SWELIB-1723,Re: SWELIB-1715 Anmeldung zu SAP Fiori funktio...,Ich könnte weinen! Habe extra den MacBook mein...,Mareen Wienand,duygukaranfil@me.com,High,Geschlossen,2021-08-24 11:50:01.894,2021-08-24 16:20:34.555,3,5
465,Support,SWELIB-1687,Re: SWELIB-1667 Generelle Fragen zu offenen Au...,"Hallo Herr Schott,\n\nich bin es noch mal. Ich...",WeLearninBits,jarugich@yahoo.de,High,Waiting for customer,2021-07-23 11:18:45.839,2021-07-28 17:31:41.881,0,5


cluster_zsmfassung                                     \
                                  count      mean       std  min  25%  50%   
cluster_beschreibung                                                         
0                                 284.0  1.200704  1.223388  0.0  0.0  2.0   
1                                  81.0  3.271605  1.457844  0.0  4.0  4.0   
2                                 683.0  0.506589  1.071300  0.0  0.0  0.0   
3                                 375.0  1.136000  1.785311  0.0  0.0  0.0   
4                                 587.0  0.086882  0.486278  0.0  0.0  0.0   
5                                  58.0  0.586207  1.214452  0.0  0.0  0.0   

                                
                      75%  max  
cluster_beschreibung            
0                     2.0  4.0  
1                     4.0  4.0  
2                     0.0  5.0  
3                     2.0  5.0  
4                     0.0  5.0  
5                     0.0  5.0

## 2. Topic Modelling with Non-negative Matrix Factorization (NMF) 
The following lines leverage the following python packages`
* `from sklearn.feature_extraction.text import TfidfVectorizer`
* `from sklearn.decomposition import NMF, LatentDirichletAllocation`

### 2.1 Topic Models auf Basis der Beschreibung

In [12]:
# Vectorize the Input Text
tfidf = TfidfVectorizer(max_features=1000, stop_words=german_stop_words)
X_tfidf = tfidf.fit_transform(beschreibung)
tfidf_feature_names = tfidf.get_feature_names_out()

#### 2.1.1. 

In [13]:
# Set number of Topics - in best case, you actually loop through 
# and have it set different numbers of topics and then look at the coherence score
no_topics = 10

# Run NMF
nmf = NMF(n_components=no_topics).fit(X_tfidf)

# Run LDA
#lda = LatentDirichletAllocation(n_components=no_topics).fit(X_tfidf)

# Run LDA with tf-idf
#lda_tf = LatentDirichletAllocation(n_components=no_topics).fit(X_tf)

In [17]:
# https://notebook.community/wickedoto/Cmpe547_Final_Project/topic-modeling-nmf-klbnmf


nmf_vis_data = pyLDAvis.sklearn.prepare(nmf, X_tfidf, tfidf_feature_names)
pyLDAvis.display(nmf_vis_data)

AttributeError: 'numpy.ndarray' object has no attribute 'get_feature_names'

### 2.2 Extraction of Topics

In [16]:
#extract topics
def display_topics(model, feature_names, no_top_words):
    topics=[]
    for topic_idx, topic in enumerate(model.components_):
        #rint ("Topic %d:" % (topic_idx))
        topic_words=" ".join([feature_names[i] for i in topic.argsort()[:-no_top_words - 1:-1]])
        #rint(topic_words)
        topics.append(topic_words)
    return topics

no_top_words = 10
#rint("NMF: ")
topics_nmf=display_topics(nmf, tfidf_feature_names, no_top_words)
#rint("\nLDA: ")
#topics_lda=display_topics(lda, tfidf_feature_names, no_top_words)

#rint(topics_nmf)
#rint(topics_lda)

#pred_lda=lda.transform(X_tfidf)
pred_nmf=nmf.transform(X_tfidf)

#res_lda=[topics_lda[np.argmax(r)] for r in pred_lda]
res_nmf=[topics_nmf[np.argmax(r)] for r in pred_nmf]

#reports['topic_lda']=res_lda
reports['topic_nmf']=res_nmf

In [17]:
topics_nmf

['kurs prüfung sap ts410 fragen möchte gerne zertifikat grüße frage',
 'color 333333 atlassian iis ls jira 3572b0 net 707070 https',
 '_x000d_ ja de_x000d_ datenschutz gelesen akzeptiert datenschutzerklärung com_x000d_ grüßen_x000d_ 4students',
 'schritt übung fallstudie angezeigt anlegen fehler fehlermeldung material screenshot nummer',
 'the to and is course have of my it for',
 'dank vielen voraus hallo grüße zusammen hilfe viele bitte leider',
 'sap passwort mehr anmelden fiori gui bitte leider anmeldung hallo',
 'test adresse frage fragen email zwischentest message prüfung antworten datenschutz',
 'geehrte damen herren freundlichen grüßen bitte leider bedanke wäre möglich',
 'guten tag grüßen freundlichen problem anbei rückmeldung photo danke lösen']

In [17]:
pred_nmf

array([[2.13520481e-02, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 3.70960279e-02],
       [5.75466162e-02, 0.00000000e+00, 1.06023625e-04, ...,
        2.74180870e-04, 4.48176757e-02, 1.50548124e-02],
       [1.23683901e-01, 0.00000000e+00, 7.97850134e-04, ...,
        2.99177547e-03, 5.88495778e-02, 6.08241998e-03],
       ...,
       [7.30165220e-02, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 3.61865609e-03, 0.00000000e+00],
       [7.83916728e-02, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 6.81419600e-02],
       [0.00000000e+00, 8.33325728e-04, 0.00000000e+00, ...,
        9.02676837e-04, 5.01087535e-03, 2.64505083e-03]])

In [20]:
res_nmf

['kurs den ich sap habe der mich prüfung noch ts410',
 'ich habe nicht kann und das wie wenn aber schritt',
 'ich habe nicht kann und das wie wenn aber schritt',
 'kurs den ich sap habe der mich prüfung noch ts410',
 'die zu ist es für das sind man werden und',
 'ich habe nicht kann und das wie wenn aber schritt',
 'der bei übung wird angezeigt mir schritt fallstudie nicht ist',
 'kurs den ich sap habe der mich prüfung noch ts410',
 'ich habe nicht kann und das wie wenn aber schritt',
 'kurs den ich sap habe der mich prüfung noch ts410',
 'nicht passwort sap kann mich anmelden mein mehr ich fiori',
 'der bei übung wird angezeigt mir schritt fallstudie nicht ist',
 'der bei übung wird angezeigt mir schritt fallstudie nicht ist',
 'sehr geehrte damen herren und freundlichen grüßen mit sie ich',
 'die zu ist es für das sind man werden und',
 'der bei übung wird angezeigt mir schritt fallstudie nicht ist',
 'kurs den ich sap habe der mich prüfung noch ts410',
 'dank vielen im voraus hallo 

In [21]:
pyLDAvis.prepare(mds='tsne', **res_nmf)

TypeError: pyLDAvis._prepare.prepare() argument after ** must be a mapping, not list

In [ ]:
np.unique(res_lda)

In [ ]:
np.unique(res_nmf)

In [ ]:
grouped=reports.groupby('topic_nmf')
for gp_name, gp in grouped:
    display(gp)

In [ ]:
#reports.to_excel("ERP EUT Topics Models Level 1.xlsx") 